# Train a Hypergraph Neural Network

In this notebook, we will create and train a two-step message passing network in the hypergraph domain. We will use a benchmark dataset, shrec16, a collection of 3D meshes, to train the model to perform classification at the level of the hypergraph. 

In [1]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split

import toponetx.datasets as datasets
from topomodelx.nn.hypergraph.template_layer import TemplateLayer

If GPU's are available, we will make use of them. Otherwise, this will run on CPU.

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Pre-processing

## Import data ##

The first step is to import the dataset, shrec 16, a benchmark dataset for 3D mesh classification. We then lift each graph into our domain of choice, a hypergraph.

We will also retrieve:
- input signal on the edges for each of these hypergraphs, as that will be what we feed the model in input
- the label associated to the hypergraph

In [3]:
shrec, _ = datasets.mesh.shrec_16(size="small")

shrec = {key: np.array(value) for key, value in shrec.items()}
x_0s = shrec["node_feat"]
x_1s = shrec["edge_feat"]
x_2s = shrec["face_feat"]

ys = shrec["label"]
simplexes = shrec["complexes"]

Loading shrec 16 small dataset...

done!


In [4]:
i_complex = 6
print(
    f"The {i_complex}th simplicial complex has {x_0s[i_complex].shape[0]} nodes with features of dimension {x_0s[i_complex].shape[1]}."
)
print(
    f"The {i_complex}th simplicial complex has {x_1s[i_complex].shape[0]} edges with features of dimension {x_1s[i_complex].shape[1]}."
)
print(
    f"The {i_complex}th simplicial complex has {x_2s[i_complex].shape[0]} faces with features of dimension {x_2s[i_complex].shape[1]}."
)

The 6th simplicial complex has 252 nodes with features of dimension 6.
The 6th simplicial complex has 750 edges with features of dimension 10.
The 6th simplicial complex has 500 faces with features of dimension 7.


## Define neighborhood structures and lift into hypergraph domain. ##

Now we retrieve the neighborhood structures (i.e. their representative matrices) that we will use to send messges on each simplicial complex. In the case of this architecture, we need the boundary matrix (or incidence matrix) $B_1$ with shape $n_\text{nodes} \times n_\text{edges}$.

Once we have recorded the incidence matrix (note that all incidence amtrices in the hypergraph domain must be unsigned), we lift each simplicial complex into a hypergraph. The pairwise edges will become pairwise hyperedges, and faces in the simplciial complex will become 3-wise hyperedges.

In [5]:
hg_list = []
incidence_1_list = []
for simplex in simplexes:
    incidence_1 = simplex.incidence_matrix(rank=1, signed=False)
    incidence_1 = torch.from_numpy(incidence_1.todense()).to_sparse()
    incidence_1_list.append(incidence_1)
    hg = simplex.to_hypergraph()
    hg_list.append(hg)

In [6]:
i_complex = 6
print(
    f"The {i_complex}th hypergraph has an incidence matrix of shape {incidence_1_list[i_complex].shape}."
)

The 6th hypergraph has an incidence matrix of shape torch.Size([252, 750]).


# Create the Neural Network

Using the TemplateLayer class, we create a neural network with stacked layers.

In [7]:
channels_edge = x_1s[0].shape[1]
channels_node = x_0s[0].shape[1]

In [8]:
class TemplateNN(torch.nn.Module):
    """Neural network implementation of Template for hypergraph classification.

    Parameters
    ---------
    channels_edge : int
        Dimension of edge features
    channels_node : int
        Dimension of node features
    n_layer : 2
        Amount of message passing layers.

    """

    def __init__(self, channels_edge, channels_node, n_layers=2):
        super().__init__()
        layers = []
        for _ in range(n_layers):
            layers.append(
                TemplateLayer(
                    in_channels=channels_edge,
                    intermediate_channels=channels_node,
                    out_channels=channels_edge,
                )
            )
        self.layers = torch.nn.ModuleList(layers)
        self.linear = torch.nn.Linear(channels_edge, 1)

    def forward(self, x_1, incidence_1):
        """Forward computation through layers, then linear layer, then global max pooling.

        Parameters
        ---------
        x_1 : tensor
            shape = [n_edges, channels_edge]
            Edge features.

        incidence_1 : tensor
            shape = [n_nodes, n_edges]
            Boundary matrix of rank 1.

        Returns
        --------
        _ : tensor
            shape = [1]
            Label assigned to whole complex.
        """
        for layer in self.layers:
            x_1 = layer(x_1, incidence_1)
        pooled_x = torch.max(x_1, dim=0)[0]
        return torch.sigmoid(self.linear(pooled_x))[0]

# Train the Neural Network

We specify the model, the loss, and an optimizer.

In [9]:
model = TemplateNN(channels_edge, channels_node, n_layers=2)
model = model.to(device)
opt = torch.optim.Adam(model.parameters(), lr=0.1)

Split the dataset into train and test sets.

In [10]:
test_size = 0.2
x_1_train, x_1_test = train_test_split(x_1s, test_size=test_size, shuffle=False)
incidence_1_train, incidence_1_test = train_test_split(
    incidence_1_list, test_size=test_size, shuffle=False
)
y_train, y_test = train_test_split(ys, test_size=test_size, shuffle=False)

The following cell performs the training, looping over the network for a low amount of epochs. We keep training minimal for the purpose of rapid testing.

In [11]:
loss_fn = torch.nn.MSELoss()

In [12]:
test_interval = 1
num_epochs = 5
for epoch_i in range(1, num_epochs + 1):
    epoch_loss = []
    model.train()
    for x_1, incidence_1, y in zip(x_1_train, incidence_1_train, y_train):
        x_1 = torch.tensor(x_1)
        x_1, incidence_1, y = (
            x_1.float().to(device),
            incidence_1.float().to(device),
            torch.tensor(y, dtype=torch.float).to(device),
        )
        opt.zero_grad()
        y_hat = model(x_1, incidence_1)
        loss = loss_fn(y_hat, y)

        loss.backward()
        opt.step()
        epoch_loss.append(loss.item())

    print(
        f"Epoch: {epoch_i} loss: {np.mean(epoch_loss):.4f}",
        flush=True,
    )
    if epoch_i % test_interval == 0:
        with torch.no_grad():
            for x_1, incidence_1, y in zip(x_1_test, incidence_1_test, y_test):
                x_1 = torch.tensor(x_1)
                x_1, incidence_1, y = (
                    x_1.float().to(device),
                    incidence_1.float().to(device),
                    torch.tensor(y, dtype=torch.float).to(device),
                )
                y_hat = model(x_1, incidence_1)
                loss = loss_fn(y_hat, y)

            print(f"Test_loss: {loss:.4f}", flush=True)

Epoch: 1 loss: 275.0353
Test_loss: 529.0022
Epoch: 2 loss: 274.6137
Test_loss: 529.0016
Epoch: 3 loss: 274.6134
Test_loss: 529.0012
Epoch: 4 loss: 274.6131
Test_loss: 529.0009
Epoch: 5 loss: 274.6130
Test_loss: 529.0007
